# Troubleshooting PRECONDITION_FAILED error

Youtube's internal API has changed and now seems to require both the `context.client.visitorData` fields and the `context.request.attestationResponseData` object:

```json
{
    "context": {
        "client": {
            "visitorData": "Cgt4YndfUjRMNDd...",
            "userAgent": "Mozilla/5.0 (X11; Linux x86_64; rv:145.0) Gecko/20100101 Firefox/145.0,gzip(gfe)",
            "clientName": "WEB",
            "clientVersion": "2.20251222.04.00"
        },
        "request": {
            "attestationResponseData": {
                "challenge": "a=6&a2=10&b=4pGJcytzFKRXtSh_mtRrrAPR6NA&c=1766585477&d=1&t=21600&c1a=1&c6a=1&c6b=1&hh=jme44dPEfzssT6Y0hd5koMdguh-8S7QszeKOHCvxKzw",
                "webResponse": "$Nqk5qfFRAAZhip..."
            }
        }
    },
    "params": "CgtkUXc0dzlXZ1hjURIOQ2dBU0FtVnVHZ0ElM0QYASozZW5nYWdlbWVudC1wYW5lbC1zZWFyY2hhYmxlLXRyYW5zY3JpcHQtc2VhcmNoLXBhbmVsMAE4AUAB",
    "externalVideoId": "dQw4w9WgXcQ"
}
```



## Testing android_sdkless with /get_transcript API

According to yt-dlp source code, `android_sdkless` client "Doesn't require a PoToken for some reason".

Let's test if this client can use the `/get_transcript` endpoint (engagement panel approach) successfully.

In [22]:
# Force reload the module
import importlib
import yt_transcript_fetcher.protobuf
importlib.reload(yt_transcript_fetcher.protobuf)

import requests
import base64
from yt_transcript_fetcher.protobuf import generate_params, encode_visitor_data

def test_get_transcript_with_client(video_id, client_name, client_info, lang="en"):
    """Test if /get_transcript API works with a given client."""
    
    # Build innertube context
    innertube_context = {"client": client_info.get("INNERTUBE_CONTEXT", {}).get("client", {}).copy()}
    
    # Add visitorData
    visitor_data = encode_visitor_data()
    innertube_context["client"]["visitorData"] = visitor_data
    
    # Request body
    body = {
        "context": innertube_context,
        "params": generate_params(video_id, lang)
    }
    
    # API endpoint
    api_url = "https://www.youtube.com/youtubei/v1/get_transcript"
    
    # Headers
    user_agent = innertube_context.get("client", {}).get("userAgent", 
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": user_agent,
        "X-Goog-Visitor-Id": visitor_data,
    }
    
    try:
        resp = requests.post(
            api_url,
            json=body,
            headers=headers,
            timeout=10
        )
        
        print(f"\n{client_name}:")
        print(f"  Status: {resp.status_code}")
        
        if resp.status_code == 200:
            data = resp.json()
            # Check for transcript actions
            actions = data.get("actions", [])
            if actions:
                update_action = actions[0].get("updateEngagementPanelAction", {})
                content = update_action.get("content", {})
                transcript_renderer = content.get("transcriptRenderer", {})
                body_content = transcript_renderer.get("content", {}).get("transcriptSearchPanelRenderer", {})
                segments = body_content.get("body", {}).get("transcriptSegmentListRenderer", {}).get("initialSegments", [])
                print(f"  Transcript segments: {len(segments)}")
                return len(segments) > 0
            else:
                print(f"  Response keys: {list(data.keys())}")
                return False
        else:
            error = resp.json() if resp.headers.get("content-type", "").startswith("application/json") else resp.text
            if isinstance(error, dict):
                err_msg = error.get("error", {}).get("message", str(error))[:100]
                err_status = error.get("error", {}).get("status", "")
                print(f"  Error: {err_status} - {err_msg}")
            else:
                print(f"  Error: {str(error)[:100]}...")
            return False
            
    except Exception as e:
        print(f"  Exception: {e}")
        return False

# Test clients that might work without PO Token
VIDEO_ID = "dQw4w9WgXcQ"

test_clients = {
    "android_sdkless": {
        "INNERTUBE_CONTEXT": {
            "client": {
                "clientName": "ANDROID",
                "clientVersion": "20.10.38",
                "userAgent": "com.google.android.youtube/20.10.38 (Linux; U; Android 11) gzip",
                "osName": "Android",
                "osVersion": "11",
            },
        },
    },
    "android": {
        "INNERTUBE_CONTEXT": {
            "client": {
                "clientName": "ANDROID",
                "clientVersion": "19.09.37",
                "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
                "osName": "Android",
                "osVersion": "11",
            },
        },
    },
    "ios": {
        "INNERTUBE_CONTEXT": {
            "client": {
                "clientName": "IOS",
                "clientVersion": "19.09.3",
                "deviceMake": "Apple",
                "deviceModel": "iPhone",
                "userAgent": "com.google.ios.youtube/19.09.3 (iPhone; U; CPU iPhone OS 17_4 like Mac OS X)",
                "osName": "iPhone",
                "osVersion": "17.4",
            },
        },
    },
    "web": {
        "INNERTUBE_CONTEXT": {
            "client": {
                "clientName": "WEB",
                "clientVersion": "2.20240101",
            },
        },
    },
}

print("Testing /get_transcript API with different clients...")
print("=" * 60)

for name, info in test_clients.items():
    result = test_get_transcript_with_client(VIDEO_ID, name, info)
    if result:
        print(f"  ✓ {name} WORKS!")
    else:
        print(f"  ✗ {name} failed")

Testing /get_transcript API with different clients...

android_sdkless:
  Status: 400
  Error: FAILED_PRECONDITION - Precondition check failed.
  ✗ android_sdkless failed

android:
  Status: 200
  Transcript segments: 0
  ✗ android failed

ios:
  Status: 200
  Transcript segments: 0
  ✗ ios failed

web:
  Status: 400
  Error: FAILED_PRECONDITION - Precondition check failed.
  ✗ web failed


In [23]:
# Let's look at what the android client actually returns
import json

def detailed_get_transcript(video_id, client_name, client_info, lang="en"):
    """Get detailed response from /get_transcript API."""
    
    innertube_context = {"client": client_info.get("INNERTUBE_CONTEXT", {}).get("client", {}).copy()}
    visitor_data = encode_visitor_data()
    innertube_context["client"]["visitorData"] = visitor_data
    
    body = {
        "context": innertube_context,
        "params": generate_params(video_id, lang)
    }
    
    api_url = "https://www.youtube.com/youtubei/v1/get_transcript"
    
    user_agent = innertube_context.get("client", {}).get("userAgent", 
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": user_agent,
        "X-Goog-Visitor-Id": visitor_data,
    }
    
    resp = requests.post(api_url, json=body, headers=headers, timeout=10)
    return resp.status_code, resp.json()

# Test with android client
status, data = detailed_get_transcript("dQw4w9WgXcQ", "android", test_clients["android"])
print(f"Android client response (status {status}):")
print(json.dumps(data, indent=2)[:2000])

Android client response (status 200):
{
  "responseContext": {
    "visitorData": "CgtZMkt6Nm1tWVVxayjn2MrKBjIKCgJHQhIEGgAgSjoMCAEggqjIv_SMq6lpWKbjmtaXu-fQHQ%3D%3D",
    "serviceTrackingParams": [
      {
        "service": "CSI",
        "params": [
          {
            "key": "c",
            "value": "ANDROID"
          },
          {
            "key": "cver",
            "value": "19.09.37"
          },
          {
            "key": "yt_li",
            "value": "0"
          },
          {
            "key": "GetVideoTranscript_rid",
            "value": "0x798c1d2ab3beffdc"
          }
        ]
      },
      {
        "service": "GFEEDBACK",
        "params": [
          {
            "key": "logged_in",
            "value": "0"
          },
          {
            "key": "visitor_data",
            "value": "CgtZMkt6Nm1tWVVxayjn2MrKBjIKCgJHQhIEGgAgSjoMCAEggqjIv_SMq6lp"
          }
        ]
      },
      {
        "service": "GUIDED_HELP",
        "params": [
          {

In [24]:
# Check what keys are in the android response
print("Top-level keys in android response:", list(data.keys()))
print()

# Check for actions
if "actions" in data:
    print("Actions found:", len(data["actions"]))
    if data["actions"]:
        print("First action keys:", list(data["actions"][0].keys()))
else:
    print("No 'actions' key - transcript not returned")
    
# Let's also try using your library's actual API class to see if it works
print("\n" + "=" * 60)
print("Testing with the library's actual API class:")
from yt_transcript_fetcher.api import YouTubeTranscriptFetcher

fetcher = YouTubeTranscriptFetcher()
try:
    languages = fetcher.list_languages("dQw4w9WgXcQ")
    print(f"Languages found: {len(languages)}")
    for lang in languages[:5]:
        print(f"  {lang}")
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")

Top-level keys in android response: ['responseContext', 'actions', 'trackingParams', 'frameworkUpdates']

Actions found: 1
First action keys: ['clickTrackingParams', 'elementsCommand']

Testing with the library's actual API class:
Languages found: 6
  English (en)
  English (auto-generated) (en)
  German (Germany) (de-DE)
  Japanese (ja)
  Portuguese (Brazil) (pt-BR)


## Summary of Findings

### The `/get_transcript` API (Engagement Panel Approach)
- **WEB, android_sdkless (newer Android)**: Return `FAILED_PRECONDITION` (400) - now requires PO Token (attestation)
- **Android (older), iOS**: Return 200 but with an `elementsCommand` action (not `updateEngagementPanelAction`) - different response format that doesn't include transcripts via engagement panels
- **TVHTML5_SIMPLY**: Returns 200 but empty (no engagement panel support)

### The `/player` API + Timedtext Approach
- **ALL tested clients work**: Successfully returns caption tracks with baseUrl
- **Timedtext fetch works**: Can fetch and parse XML transcript segments
- **Downside**: Rate limiting concerns (requires proxy rotation for production)

### Options:
1. **Switch to `/player` API + Timedtext** - Works but has rate limiting
2. **Implement PO Token generation** - Requires JavaScript runtime (bgutils-js)
3. **Use a PO Token provider service** - External dependency
4. **Accept current limitation** - Only works with proper attestation

In [25]:
# Let's explore the 'elementsCommand' from the Android client to see if it contains transcript data
print("Exploring Android client response structure...")
print("=" * 60)

status, data = detailed_get_transcript("dQw4w9WgXcQ", "android", test_clients["android"])

if "actions" in data and data["actions"]:
    action = data["actions"][0]
    print(f"Action keys: {list(action.keys())}")
    
    if "elementsCommand" in action:
        elements_cmd = action["elementsCommand"]
        print(f"\nelementsCommand keys: {list(elements_cmd.keys())}")
        
        # See the structure
        import json
        print("\nelementsCommand content (truncated):")
        print(json.dumps(elements_cmd, indent=2)[:3000])

Exploring Android client response structure...


Action keys: ['clickTrackingParams', 'elementsCommand']

elementsCommand keys: ['transformEntityCommand']

elementsCommand content (truncated):
{
  "transformEntityCommand": {
    "identifier": "dQw4w9WgXcQ.transcript.full.state.key",
    "transform": {
      "types": [
        {
          "typeId": 2,
          "fieldType": "EKO_FIELD_TYPE_MESSAGE"
        },
        {
          "typeId": 12,
          "fieldType": "EKO_FIELD_TYPE_INT32"
        }
      ],
      "variables": [
        {
          "variableId": 1,
          "variableType": "EKO_VARIABLE_TYPE_INPUT"
        },
        {
          "variableId": 2,
          "typeId": 2,
          "variableType": "EKO_VARIABLE_TYPE_OUTPUT",
          "value": {
            "messageValue": {
              "fields": [
                {
                  "tag": 1,
                  "value": {
                    "chooseValue": {
                      "whenThenValues": [
                        {
                          "whenValue": {
     

In [26]:
# Explore the elementsCommand path for transcripts!
print("Exploring Android client elementsCommand for transcripts...")
print("=" * 60)

status, data = detailed_get_transcript("dQw4w9WgXcQ", "android", test_clients["android"])

if "actions" in data and data["actions"]:
    action = data["actions"][0]
    
    try:
        # Navigate to the transcript segments
        elements_cmd = action.get("elementsCommand", {})
        transform_cmd = elements_cmd.get("transformEntityCommand", {})
        arguments = transform_cmd.get("arguments", {})
        transform_args = arguments.get("transformTranscriptSegmentListArguments", {})
        overwrite = transform_args.get("overwrite", {})
        initial_segments = overwrite.get("initialSegments", [])
        
        print(f"Found {len(initial_segments)} transcript segments!")
        
        if initial_segments:
            print("\nFirst 5 segments:")
            for i, seg in enumerate(initial_segments[:5]):
                print(f"\nSegment {i+1}:")
                print(json.dumps(seg, indent=2)[:500])
    except Exception as e:
        print(f"Error navigating structure: {e}")
        
        # Let's try to find the path
        print("\nTrying to find the path...")
        elements_cmd = action.get("elementsCommand", {})
        transform_cmd = elements_cmd.get("transformEntityCommand", {})
        print(f"transformEntityCommand keys: {list(transform_cmd.keys())}")

Exploring Android client elementsCommand for transcripts...
Found 52 transcript segments!

First 5 segments:

Segment 1:
{
  "transcriptSegmentRenderer": {
    "startMs": "320",
    "endMs": "14580",
    "snippet": {
      "elementsAttributedString": {
        "content": "[Music]"
      }
    },
    "startTimeText": {
      "elementsAttributedString": {
        "content": "0:00"
      }
    },
    "trackingParams": "CDQQ0_YHGDYiEwiH-L75m-ORAxV7ax4CHa1iL78=",
    "accessibility": {
      "accessibilityData": {
        "label": "0 seconds [Music]"
      }
    },
    "entityKey": "EhVkUXc0dzlXZ1hjUV8zMjBfMTQ1ODAgngIo

Segment 2:
{
  "transcriptSegmentRenderer": {
    "startMs": "18800",
    "endMs": "21800",
    "snippet": {
      "elementsAttributedString": {
        "content": "We're no strangers to"
      }
    },
    "startTimeText": {
      "elementsAttributedString": {
        "content": "0:18"
      }
    },
    "trackingParams": "CDMQ0_YHGDciEwiH-L75m-ORAxV7ax4CHa1iL78=",
    "acc

In [27]:
# Parse the Android client elementsCommand response format
def parse_elements_command_transcript(response_data):
    """Parse transcript from Android client's elementsCommand response."""
    segments = []
    
    try:
        actions = response_data.get("actions", [])
        if not actions:
            return segments
        
        action = actions[0]
        elements_cmd = action.get("elementsCommand", {})
        transform_cmd = elements_cmd.get("transformEntityCommand", {})
        arguments = transform_cmd.get("arguments", {})
        transform_args = arguments.get("transformTranscriptSegmentListArguments", {})
        overwrite = transform_args.get("overwrite", {})
        initial_segments = overwrite.get("initialSegments", [])
        
        for seg in initial_segments:
            renderer = seg.get("transcriptSegmentRenderer", {})
            
            start_ms = int(renderer.get("startMs", 0))
            end_ms = int(renderer.get("endMs", 0))
            text = renderer.get("snippet", {}).get("elementsAttributedString", {}).get("content", "")
            
            if text:
                segments.append({
                    "start": start_ms / 1000,
                    "duration": (end_ms - start_ms) / 1000,
                    "text": text
                })
        
        return segments
    except Exception as e:
        print(f"Error parsing elementsCommand transcript: {e}")
        return segments

# Test the parser
status, data = detailed_get_transcript("dQw4w9WgXcQ", "android", test_clients["android"])
segments = parse_elements_command_transcript(data)

print(f"Parsed {len(segments)} transcript segments from Android client!")
print("\nFirst 10 segments:")
for seg in segments[:10]:
    print(f"  [{seg['start']:.2f}s - {seg['start'] + seg['duration']:.2f}s] {seg['text']}")

print("\n" + "=" * 60)
print("SUCCESS! The older Android client (19.09.37) works without attestation!")
print("=" * 60)

Parsed 52 transcript segments from Android client!

First 10 segments:
  [0.32s - 14.58s] [Music]
  [18.80s - 21.80s] We're no strangers to
  [21.80s - 25.96s] love. You know the rules and so do
  [25.96s - 29.12s] I. I feel commitments from what I'm
  [29.12s - 30.28s] thinking
  [30.28s - 34.36s] of. You wouldn't get this from any other
  [34.36s - 39.56s] guy. I just want to tell you how I'm
  [39.56s - 43.12s] feeling. Got to make you understand.
  [43.12s - 45.84s] Never going to give you up. I'm going to
  [45.84s - 49.20s] let you down. I'm going to run around

SUCCESS! The older Android client (19.09.37) works without attestation!


In [28]:
# Test iOS client too
print("Testing iOS client with older version...")
print("=" * 60)

ios_old = {
    "INNERTUBE_CONTEXT": {
        "client": {
            "clientName": "IOS",
            "clientVersion": "19.09.3",  # Older version
            "deviceMake": "Apple",
            "deviceModel": "iPhone",
            "userAgent": "com.google.ios.youtube/19.09.3 (iPhone; U; CPU iPhone OS 17_4 like Mac OS X)",
            "osName": "iPhone",
            "osVersion": "17.4",
        },
    },
}

status, data = detailed_get_transcript("dQw4w9WgXcQ", "ios", ios_old)
print(f"iOS client response status: {status}")

if status == 200:
    segments = parse_elements_command_transcript(data)
    print(f"Parsed {len(segments)} transcript segments from iOS client!")
    if segments:
        print("\nFirst 5 segments:")
        for seg in segments[:5]:
            print(f"  [{seg['start']:.2f}s] {seg['text']}")
else:
    print(f"iOS client failed with status {status}")

Testing iOS client with older version...


iOS client response status: 200
Parsed 52 transcript segments from iOS client!

First 5 segments:
  [0.32s] [Music]
  [18.80s] We're no strangers to
  [21.80s] love. You know the rules and so do
  [25.96s] I. I feel commitments from what I'm
  [29.12s] thinking


## 🎉 SOLUTION FOUND!

### Working Clients (No Attestation Required)
The **older mobile client versions** (Android 19.09.37, iOS 19.09.3) return transcripts via the `/get_transcript` API without requiring attestation (PO Token).

### Key Differences
| Client Version | Response Format | Works? |
|---------------|----------------|--------|
| Android 20.10.38 (android_sdkless) | `FAILED_PRECONDITION` | ❌ |
| Android 19.09.37 | `elementsCommand.transformEntityCommand.arguments.transformTranscriptSegmentListArguments` | ✅ |
| iOS 19.09.3 | Same as Android 19.x | ✅ |
| WEB | `FAILED_PRECONDITION` | ❌ |

### Fix Required
1. **Downgrade** the Android client version from `20.10.38` to `19.09.37`
2. **Parse** the new response format: `elementsCommand` → `transformEntityCommand` → `arguments` → `transformTranscriptSegmentListArguments` → `overwrite` → `initialSegments`
3. Each segment has `transcriptSegmentRenderer` with:
   - `startMs` / `endMs` - timestamps in milliseconds
   - `snippet.elementsAttributedString.content` - the text

In [29]:
import pprint
# Let's explore the Android response for language menu data
print("Exploring Android response for language menu...")
print("=" * 60)

status, data = detailed_get_transcript("dQw4w9WgXcQ", "android", test_clients["android"])

# Look for language-related keys in the response
def find_keys_recursive(obj, target_keys, path="", results=None):
    if results is None:
        results = []
    if isinstance(obj, dict):
        for key, value in obj.items():
            current_path = f"{path}.{key}" if path else key
            if any(k.lower() in key.lower() for k in target_keys):
                results.append((current_path, type(value).__name__))
            find_keys_recursive(value, target_keys, current_path, results)
    elif isinstance(obj, list):
        for i, item in enumerate(obj):
            find_keys_recursive(item, target_keys, f"{path}[{i}]", results)
    return results

# Search for language-related keys
language_keys = find_keys_recursive(data, ["language", "lang", "menu", "caption"])
print("Found language-related keys:")
for path, type_name in language_keys[:20]:
    print(f"  {path} ({type_name})")

# Check if there's a frameworkUpdates section
if "frameworkUpdates" in data:
    print("\n\nframeworkUpdates keys:")
    fu = data["frameworkUpdates"]
    pprint.pprint(fu, indent=2)

Exploring Android response for language menu...


Found language-related keys:


frameworkUpdates keys:
{ 'entityBatchUpdate': { 'mutations': [ { 'entityKey': 'EgtkUXc0dzlXZ1hjUSCgAigB',
                                          'payload': { 'transcriptSegmentsDataEntity': { 'key': 'EgtkUXc0dzlXZ1hjUSCgAigB',
                                                                                         'segmentsData': [ { 'endMs': '14580',
                                                                                                             'scrollCommand': { 'clickTrackingParams': 'CAAQw7wCIhMIhr3W-ZvjkQMVJkT2CB1y7SMAygEEcsZulw==',
                                                                                                                                'elementsCommand': { 'collectionTypeScrollToItemCommand': { 'animationConfig': { 'enableAnimation': False},
                                                                                                                                                                             

In [30]:
# Test with "xx" language (invalid) to see if we get a language list
# This is how the library's list_languages works
print("Testing Android with 'xx' language to see language list...")
print("=" * 60)

from yt_transcript_fetcher.protobuf import generate_params, encode_visitor_data

android_old = {
    "INNERTUBE_CONTEXT": {
        "client": {
            "clientName": "ANDROID",
            "clientVersion": "19.09.37",
            "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
            "osName": "Android",
            "osVersion": "11",
        },
    },
}

innertube_context = {"client": android_old["INNERTUBE_CONTEXT"]["client"].copy()}
visitor_data = encode_visitor_data()
innertube_context["client"]["visitorData"] = visitor_data

body = {
    "context": innertube_context,
    "params": generate_params("dQw4w9WgXcQ", "xx"),  # "xx" is invalid
    "externalVideoId": "dQw4w9WgXcQ",
}

api_url = "https://www.youtube.com/youtubei/v1/get_transcript"

headers = {
    "Content-Type": "application/json",
    "User-Agent": android_old["INNERTUBE_CONTEXT"]["client"]["userAgent"],
    "X-Goog-Visitor-Id": visitor_data,
}

resp = requests.post(api_url, json=body, headers=headers, timeout=10)
print(f"Status: {resp.status_code}")

if resp.status_code == 200:
    data = resp.json()
    print(f"Top-level keys: {list(data.keys())}")
    
    # Look for any language-related info in actions
    if "actions" in data and data["actions"]:
        print(f"Actions count: {len(data['actions'])}")
        action = data["actions"][0]
        print(f"Action keys: {list(action.keys())}")
        
        # Search for language menu in the elementsCommand structure
        if "elementsCommand" in action:
            ec = action["elementsCommand"]
            print(f"elementsCommand keys: {list(ec.keys())}")
else:
    print(f"Error: {resp.text[:500]}")

Testing Android with 'xx' language to see language list...


Status: 200
Top-level keys: ['responseContext', 'actions', 'trackingParams', 'frameworkUpdates']
Actions count: 1
Action keys: ['clickTrackingParams', 'elementsCommand']
elementsCommand keys: ['transformEntityCommand']


In [31]:
# Let's explore the full Android response structure to find language data
print("Looking for language selection data in Android response...")
print("=" * 60)

# Look at the full action structure
action = data["actions"][0]
transform_cmd = action.get("elementsCommand", {}).get("transformEntityCommand", {})
print(f"transformEntityCommand keys: {list(transform_cmd.keys())}")

# Check if there are additional actions
if len(data["actions"]) > 1:
    print(f"\nThere are {len(data['actions'])} actions")
    for i, act in enumerate(data["actions"]):
        print(f"  Action {i}: {list(act.keys())}")

# The language info might be in frameworkUpdates or responseContext
print("\nresponseContext keys:", list(data.get("responseContext", {}).keys()))

# Check the arguments structure
arguments = transform_cmd.get("arguments", {})
print(f"\ntransformEntityCommand.arguments keys: {list(arguments.keys())}")

# Let's check if there's a language selection in transformTranscriptSegmentListArguments
transform_args = arguments.get("transformTranscriptSegmentListArguments", {})
print(f"transformTranscriptSegmentListArguments keys: {list(transform_args.keys())}")

# Check the overwrite structure for language menu
overwrite = transform_args.get("overwrite", {})
print(f"overwrite keys: {list(overwrite.keys())}")

Looking for language selection data in Android response...
transformEntityCommand keys: ['identifier', 'transform', 'arguments']

responseContext keys: ['visitorData', 'serviceTrackingParams']

transformEntityCommand.arguments keys: ['transformTranscriptSegmentListArguments']
transformTranscriptSegmentListArguments keys: ['overwrite']
overwrite keys: []


## Hybrid Approach: `/player` API for Languages + `/get_transcript` for Transcripts

Since the Android `/get_transcript` response doesn't include a language menu, we need to use the `/player` API to get available caption tracks (languages), then use `/get_transcript` with the Android client to fetch the actual transcript segments.

In [32]:
# Step 1: Get caption tracks from /player API
import requests
import copy
from urllib.parse import unquote

def get_caption_tracks_from_player(video_id, client_info=None):
    """
    Get available caption tracks (languages) from the /player API.
    
    Returns list of caption track dicts with:
    - languageCode: e.g., "en", "es", "fr"
    - name: Display name like "English" or "English (auto-generated)"
    - kind: "asr" for auto-generated, None/empty for manual
    - baseUrl: URL to fetch timedtext (we won't use this)
    """
    if client_info is None:
        # Use Android client for consistency
        client_info = {
            "INNERTUBE_CONTEXT": {
                "client": {
                    "clientName": "ANDROID",
                    "clientVersion": "19.09.37",
                    "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
                    "osName": "Android",
                    "osVersion": "11",
                },
            },
            "INNERTUBE_CONTEXT_CLIENT_NAME": 3,
        }
    
    context = copy.deepcopy(client_info.get("INNERTUBE_CONTEXT", {"client": client_info}))
    
    # Get visitor data
    visitor_data = fetch_visitor_data(context.get("client", {}))
    if visitor_data:
        context["client"]["visitorData"] = unquote(visitor_data)
    
    payload = {
        "context": context,
        "videoId": video_id,
        "playbackContext": {
            "contentPlaybackContext": {
                "html5Preference": "HTML5_PREF_WANTS",
            }
        },
        "contentCheckOk": True,
        "racyCheckOk": True
    }
    
    url = "https://www.youtube.com/youtubei/v1/player?prettyPrint=false"
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": context.get("client", {}).get("userAgent", "Mozilla/5.0"),
    }
    
    if "INNERTUBE_CONTEXT_CLIENT_NAME" in client_info:
        headers["X-Youtube-Client-Name"] = str(client_info["INNERTUBE_CONTEXT_CLIENT_NAME"])
    if "clientVersion" in context.get("client", {}):
        headers["X-Youtube-Client-Version"] = context["client"]["clientVersion"]
    
    response = requests.post(url, json=payload, headers=headers, timeout=15)
    response.raise_for_status()
    
    player_response = response.json()
    
    # Extract caption tracks
    captions = player_response.get("captions", {})
    pctr = captions.get("playerCaptionsTracklistRenderer", {})
    caption_tracks = pctr.get("captionTracks", [])
    
    return caption_tracks

# Test it
video_id = "dQw4w9WgXcQ"
tracks = get_caption_tracks_from_player(video_id)

print(f"Found {len(tracks)} caption tracks for video {video_id}:")
print("=" * 60)
for track in tracks:
    lang_code = track.get("languageCode", "")
    name = track.get("name", {}).get("simpleText", "Unknown")
    kind = track.get("kind", "")
    is_auto = kind == "asr"
    print(f"  [{lang_code}] {name} {'(auto-generated)' if is_auto else ''}")

Found 6 caption tracks for video dQw4w9WgXcQ:
  [en] Unknown 
  [en] Unknown (auto-generated)
  [de-DE] Unknown 
  [ja] Unknown 
  [pt-BR] Unknown 
  [es-419] Unknown 


In [33]:
# Let's look at the raw track data to understand the name structure
print("Raw caption track structure:")
for i, track in enumerate(tracks):
    print(f"\nTrack {i+1}:")
    print(f"  languageCode: {track.get('languageCode')}")
    print(f"  name: {track.get('name')}")
    print(f"  kind: {track.get('kind', '(none)')}")
    print(f"  vssId: {track.get('vssId', '(none)')}")

Raw caption track structure:

Track 1:
  languageCode: en
  name: {'runs': [{'text': 'English'}]}
  kind: (none)
  vssId: .en

Track 2:
  languageCode: en
  name: {'runs': [{'text': 'English (auto-generated)'}]}
  kind: asr
  vssId: a.en

Track 3:
  languageCode: de-DE
  name: {'runs': [{'text': 'German (Germany)'}]}
  kind: (none)
  vssId: .de-DE

Track 4:
  languageCode: ja
  name: {'runs': [{'text': 'Japanese'}]}
  kind: (none)
  vssId: .ja

Track 5:
  languageCode: pt-BR
  name: {'runs': [{'text': 'Portuguese (Brazil)'}]}
  kind: (none)
  vssId: .pt-BR

Track 6:
  languageCode: es-419
  name: {'runs': [{'text': 'Spanish (Latin America)'}]}
  kind: (none)
  vssId: .es-419


In [34]:
# Step 2: Create Language objects from caption tracks
from yt_transcript_fetcher.protobuf import generate_params

def extract_language_name(name_obj):
    """Extract language name from caption track name object."""
    if isinstance(name_obj, dict):
        # Check for runs format
        runs = name_obj.get("runs", [])
        if runs:
            return runs[0].get("text", "Unknown")
        # Check for simpleText format
        return name_obj.get("simpleText", "Unknown")
    return str(name_obj)

def caption_tracks_to_languages(tracks, video_id):
    """
    Convert caption tracks from /player API to Language-like objects.
    
    Each Language object needs:
    - language_code: e.g., "en"
    - language_name: e.g., "English"
    - is_auto_generated: True for ASR captions
    - params: continuation token for /get_transcript
    """
    languages = []
    
    for track in tracks:
        lang_code = track.get("languageCode", "")
        name = extract_language_name(track.get("name", {}))
        is_auto = track.get("kind") == "asr"
        
        # Generate params for /get_transcript API
        params = generate_params(video_id, lang_code)
        
        languages.append({
            "language_code": lang_code,
            "language_name": name,
            "is_auto_generated": is_auto,
            "params": params,
        })
    
    return languages

# Test conversion
video_id = "dQw4w9WgXcQ"
languages = caption_tracks_to_languages(tracks, video_id)

print(f"Converted {len(languages)} languages:")
print("=" * 60)
for lang in languages:
    auto_tag = " (auto)" if lang["is_auto_generated"] else ""
    print(f"  [{lang['language_code']}] {lang['language_name']}{auto_tag}")
    print(f"      params: {lang['params'][:50]}...")

Converted 6 languages:
  [en] English
      params: CgtkUXc0dzlXZ1hjURISQ2dOaGMzSVNBbVZ1R2dBJTNEGAEqM2...
  [en] English (auto-generated) (auto)
      params: CgtkUXc0dzlXZ1hjURISQ2dOaGMzSVNBbVZ1R2dBJTNEGAEqM2...
  [de-DE] German (Germany)
      params: CgtkUXc0dzlXZ1hjURIWQ2dOaGMzSVNCV1JsTFVSRkdnQSUzRB...
  [ja] Japanese
      params: CgtkUXc0dzlXZ1hjURISQ2dOaGMzSVNBbXBoR2dBJTNEGAEqM2...
  [pt-BR] Portuguese (Brazil)
      params: CgtkUXc0dzlXZ1hjURIWQ2dOaGMzSVNCWEIwTFVKU0dnQSUzRB...
  [es-419] Spanish (Latin America)
      params: CgtkUXc0dzlXZ1hjURIUQ2dOaGMzSVNCbVZ6TFRReE9Sb0EYAS...


In [35]:
# Step 3: Fetch transcript using Android client with /get_transcript API
from yt_transcript_fetcher.protobuf import encode_visitor_data

def fetch_transcript_android(video_id, language_code="en"):
    """
    Fetch transcript using Android client (19.09.37) which doesn't require attestation.
    Returns list of segment dicts with start, duration, text.
    """
    android_client = {
        "clientName": "ANDROID",
        "clientVersion": "19.09.37",
        "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
        "osName": "Android",
        "osVersion": "11",
    }
    
    visitor_data = encode_visitor_data()
    context = {
        "client": {
            **android_client,
            "visitorData": visitor_data,
        }
    }
    
    body = {
        "context": context,
        "params": generate_params(video_id, language_code),
    }
    
    api_url = "https://www.youtube.com/youtubei/v1/get_transcript"
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": android_client["userAgent"],
        "X-Goog-Visitor-Id": visitor_data,
    }
    
    response = requests.post(api_url, json=body, headers=headers, timeout=10)
    response.raise_for_status()
    
    data = response.json()
    
    # Parse the elementsCommand response format
    segments = []
    try:
        actions = data.get("actions", [])
        if not actions:
            return segments
        
        action = actions[0]
        elements_cmd = action.get("elementsCommand", {})
        transform_cmd = elements_cmd.get("transformEntityCommand", {})
        arguments = transform_cmd.get("arguments", {})
        transform_args = arguments.get("transformTranscriptSegmentListArguments", {})
        overwrite = transform_args.get("overwrite", {})
        initial_segments = overwrite.get("initialSegments", [])
        
        for seg in initial_segments:
            renderer = seg.get("transcriptSegmentRenderer", {})
            
            start_ms = int(renderer.get("startMs", 0))
            end_ms = int(renderer.get("endMs", 0))
            text = renderer.get("snippet", {}).get("elementsAttributedString", {}).get("content", "")
            
            if text:
                segments.append({
                    "start": start_ms / 1000,
                    "duration": (end_ms - start_ms) / 1000,
                    "text": text
                })
        
        return segments
    except Exception as e:
        print(f"Error parsing transcript: {e}")
        return segments

# Test fetching English transcript
print("Fetching English transcript...")
print("=" * 60)
segments = fetch_transcript_android("dQw4w9WgXcQ", "en")

print(f"Got {len(segments)} segments!")
print("\nFirst 10 segments:")
for seg in segments[:10]:
    print(f"  [{seg['start']:6.2f}s] {seg['text']}")

Fetching English transcript...
Got 52 segments!

First 10 segments:
  [  0.32s] [Music]
  [ 18.80s] We're no strangers to
  [ 21.80s] love. You know the rules and so do
  [ 25.96s] I. I feel commitments from what I'm
  [ 29.12s] thinking
  [ 30.28s] of. You wouldn't get this from any other
  [ 34.36s] guy. I just want to tell you how I'm
  [ 39.56s] feeling. Got to make you understand.
  [ 43.12s] Never going to give you up. I'm going to
  [ 45.84s] let you down. I'm going to run around


In [36]:
# Step 4: Test fetching a different language (Japanese)
print("Fetching Japanese transcript...")
print("=" * 60)
segments_ja = fetch_transcript_android("dQw4w9WgXcQ", "ja")

print(f"Got {len(segments_ja)} segments!")
print("\nFirst 10 segments:")
for seg in segments_ja[:10]:
    print(f"  [{seg['start']:6.2f}s] {seg['text']}")

Fetching Japanese transcript...
Got 0 segments!

First 10 segments:


In [37]:
# Debug: Let's see what the Japanese response looks like
import json

def get_raw_transcript_response(video_id, language_code):
    """Get raw response from /get_transcript for debugging."""
    android_client = {
        "clientName": "ANDROID",
        "clientVersion": "19.09.37",
        "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
        "osName": "Android",
        "osVersion": "11",
    }
    
    visitor_data = encode_visitor_data()
    context = {
        "client": {
            **android_client,
            "visitorData": visitor_data,
        }
    }
    
    body = {
        "context": context,
        "params": generate_params(video_id, language_code),
    }
    
    api_url = "https://www.youtube.com/youtubei/v1/get_transcript"
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": android_client["userAgent"],
        "X-Goog-Visitor-Id": visitor_data,
    }
    
    response = requests.post(api_url, json=body, headers=headers, timeout=10)
    return response.status_code, response.json()

# Get Japanese response
status, data = get_raw_transcript_response("dQw4w9WgXcQ", "ja")
print(f"Status: {status}")
print(f"Top-level keys: {list(data.keys())}")

if "actions" in data:
    actions = data["actions"]
    print(f"Number of actions: {len(actions)}")
    if actions:
        print(f"Action keys: {list(actions[0].keys())}")
        
        # Check if it's elementsCommand or something else
        if "elementsCommand" in actions[0]:
            ec = actions[0]["elementsCommand"]
            print(f"elementsCommand keys: {list(ec.keys())}")
            
            tc = ec.get("transformEntityCommand", {})
            args = tc.get("arguments", {})
            tsla = args.get("transformTranscriptSegmentListArguments", {})
            overwrite = tsla.get("overwrite", {})
            segments = overwrite.get("initialSegments", [])
            print(f"Segments found: {len(segments)}")
            
            if not segments:
                # Maybe it's in a different path?
                print("\nFull elementsCommand structure:")
                print(json.dumps(ec, indent=2)[:2000])

Status: 200
Top-level keys: ['responseContext', 'actions', 'trackingParams', 'frameworkUpdates']
Number of actions: 1
Action keys: ['clickTrackingParams', 'elementsCommand']
elementsCommand keys: ['transformEntityCommand']
Segments found: 0

Full elementsCommand structure:
{
  "transformEntityCommand": {
    "identifier": "dQw4w9WgXcQ.transcript.full.state.key",
    "transform": {
      "types": [
        {
          "typeId": 2,
          "fieldType": "EKO_FIELD_TYPE_MESSAGE"
        },
        {
          "typeId": 12,
          "fieldType": "EKO_FIELD_TYPE_INT32"
        }
      ],
      "variables": [
        {
          "variableId": 1,
          "variableType": "EKO_VARIABLE_TYPE_INPUT"
        },
        {
          "variableId": 2,
          "typeId": 2,
          "variableType": "EKO_VARIABLE_TYPE_OUTPUT",
          "value": {
            "messageValue": {
              "fields": [
                {
                  "tag": 1,
                  "value": {
                    "

In [38]:
# The generate_params function has an auto_generated parameter!
# Japanese captions are NOT auto-generated (no 'asr' kind)
# Let's test with auto_generated=False

def fetch_transcript_android_v2(video_id, language_code="en", auto_generated=True):
    """
    Fetch transcript using Android client (19.09.37) which doesn't require attestation.
    Now with auto_generated parameter!
    """
    android_client = {
        "clientName": "ANDROID",
        "clientVersion": "19.09.37",
        "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
        "osName": "Android",
        "osVersion": "11",
    }
    
    visitor_data = encode_visitor_data()
    context = {
        "client": {
            **android_client,
            "visitorData": visitor_data,
        }
    }
    
    # Use auto_generated parameter!
    params = generate_params(video_id, language_code, auto_generated=auto_generated)
    
    body = {
        "context": context,
        "params": params,
    }
    
    api_url = "https://www.youtube.com/youtubei/v1/get_transcript"
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": android_client["userAgent"],
        "X-Goog-Visitor-Id": visitor_data,
    }
    
    response = requests.post(api_url, json=body, headers=headers, timeout=10)
    response.raise_for_status()
    
    data = response.json()
    
    # Parse the elementsCommand response format
    segments = []
    try:
        actions = data.get("actions", [])
        if not actions:
            return segments
        
        action = actions[0]
        elements_cmd = action.get("elementsCommand", {})
        transform_cmd = elements_cmd.get("transformEntityCommand", {})
        arguments = transform_cmd.get("arguments", {})
        transform_args = arguments.get("transformTranscriptSegmentListArguments", {})
        overwrite = transform_args.get("overwrite", {})
        initial_segments = overwrite.get("initialSegments", [])
        
        for seg in initial_segments:
            renderer = seg.get("transcriptSegmentRenderer", {})
            
            start_ms = int(renderer.get("startMs", 0))
            end_ms = int(renderer.get("endMs", 0))
            text = renderer.get("snippet", {}).get("elementsAttributedString", {}).get("content", "")
            
            if text:
                segments.append({
                    "start": start_ms / 1000,
                    "duration": (end_ms - start_ms) / 1000,
                    "text": text
                })
        
        return segments
    except Exception as e:
        print(f"Error parsing transcript: {e}")
        return segments

# Test: Japanese with auto_generated=False (it's a manual caption)
print("Testing Japanese with auto_generated=False...")
print("=" * 60)
segments_ja = fetch_transcript_android_v2("dQw4w9WgXcQ", "ja", auto_generated=False)
print(f"Got {len(segments_ja)} segments!")

if segments_ja:
    print("\nFirst 10 segments:")
    for seg in segments_ja[:10]:
        print(f"  [{seg['start']:6.2f}s] {seg['text']}")
else:
    print("Still no segments - let's check what kinds are available...")
    for track in tracks:
        lang = track.get("languageCode", "")
        kind = track.get("kind", "(manual)")
        print(f"  {lang}: kind={kind}")

Testing Japanese with auto_generated=False...
Got 60 segments!

First 10 segments:
  [ 18.64s] 僕らは恋愛初心者じゃない
  [ 22.64s] ルールは互いに分かってる
  [ 27.04s] 君に全部
捧げても構わない
  [ 31.12s] 他の男は
こんなに尽くせないよ
  [ 35.16s] 僕の気持ちを
君に伝えたいんだ
  [ 40.52s] 君に分かってほしい
  [ 43.00s] 君を決して諦めない
  [ 45.20s] 決してがっかりさせない
  [ 47.32s] 言い訳したり
逃げたりしない
  [ 51.48s] 泣かせたりしない


## ✅ Complete Hybrid Solution

The solution requires:
1. **`/player` API** - Get caption tracks (languages) with their `kind` (asr vs manual)
2. **`/get_transcript` API with Android 19.09.37** - Fetch transcript segments
3. **Use `auto_generated` param correctly** - Based on `kind == "asr"` from caption tracks

The key insight is that `generate_params(video_id, lang, auto_generated=True)` adds "asr" to Field 1 of the nested protobuf. This must match the caption type!

In [39]:
# Complete Hybrid Implementation
# ==============================
# This is the complete solution that:
# 1. Uses /player API to get available languages
# 2. Uses /get_transcript with Android client to fetch transcripts

class HybridTranscriptFetcher:
    """Hybrid transcript fetcher using /player for languages and /get_transcript for content."""
    
    ANDROID_CLIENT = {
        "clientName": "ANDROID",
        "clientVersion": "19.09.37",  # Older version - no attestation required!
        "userAgent": "com.google.android.youtube/19.09.37 (Linux; U; Android 11) gzip",
        "osName": "Android",
        "osVersion": "11",
    }
    
    def __init__(self):
        self.session = requests.Session()
    
    def list_languages(self, video_id):
        """Get available caption languages using /player API."""
        visitor_data = encode_visitor_data()
        
        context = {
            "client": {
                **self.ANDROID_CLIENT,
                "visitorData": visitor_data,
            }
        }
        
        payload = {
            "context": context,
            "videoId": video_id,
            "playbackContext": {"contentPlaybackContext": {"html5Preference": "HTML5_PREF_WANTS"}},
            "contentCheckOk": True,
            "racyCheckOk": True
        }
        
        headers = {
            "Content-Type": "application/json",
            "User-Agent": self.ANDROID_CLIENT["userAgent"],
            "X-Youtube-Client-Name": "3",
            "X-Youtube-Client-Version": self.ANDROID_CLIENT["clientVersion"],
        }
        
        response = self.session.post(
            "https://www.youtube.com/youtubei/v1/player",
            json=payload,
            headers=headers,
            timeout=15
        )
        response.raise_for_status()
        
        data = response.json()
        captions = data.get("captions", {})
        pctr = captions.get("playerCaptionsTracklistRenderer", {})
        tracks = pctr.get("captionTracks", [])
        
        languages = []
        for track in tracks:
            lang_code = track.get("languageCode", "")
            name_obj = track.get("name", {})
            name = name_obj.get("runs", [{}])[0].get("text", "") if "runs" in name_obj else name_obj.get("simpleText", "")
            is_auto = track.get("kind") == "asr"
            
            languages.append({
                "language_code": lang_code,
                "language_name": name,
                "is_auto_generated": is_auto,
            })
        
        return languages
    
    def get_transcript(self, video_id, language_code="en", auto_generated=True):
        """Fetch transcript using /get_transcript with Android client."""
        visitor_data = encode_visitor_data()
        
        context = {
            "client": {
                **self.ANDROID_CLIENT,
                "visitorData": visitor_data,
            }
        }
        
        body = {
            "context": context,
            "params": generate_params(video_id, language_code, auto_generated=auto_generated),
        }
        
        headers = {
            "Content-Type": "application/json",
            "User-Agent": self.ANDROID_CLIENT["userAgent"],
            "X-Goog-Visitor-Id": visitor_data,
        }
        
        response = self.session.post(
            "https://www.youtube.com/youtubei/v1/get_transcript",
            json=body,
            headers=headers,
            timeout=10
        )
        response.raise_for_status()
        
        data = response.json()
        return self._parse_android_response(data)
    
    def _parse_android_response(self, data):
        """Parse the elementsCommand response format from Android client."""
        segments = []
        
        actions = data.get("actions", [])
        if not actions:
            return segments
        
        action = actions[0]
        elements_cmd = action.get("elementsCommand", {})
        transform_cmd = elements_cmd.get("transformEntityCommand", {})
        arguments = transform_cmd.get("arguments", {})
        transform_args = arguments.get("transformTranscriptSegmentListArguments", {})
        overwrite = transform_args.get("overwrite", {})
        initial_segments = overwrite.get("initialSegments", [])
        
        for seg in initial_segments:
            renderer = seg.get("transcriptSegmentRenderer", {})
            start_ms = int(renderer.get("startMs", 0))
            end_ms = int(renderer.get("endMs", 0))
            text = renderer.get("snippet", {}).get("elementsAttributedString", {}).get("content", "")
            
            if text:
                segments.append({
                    "start": start_ms / 1000,
                    "duration": (end_ms - start_ms) / 1000,
                    "text": text
                })
        
        return segments


# Test the complete solution
print("Testing Complete Hybrid Solution")
print("=" * 60)

fetcher = HybridTranscriptFetcher()

# 1. List available languages
print("\n1. Listing available languages...")
languages = fetcher.list_languages("dQw4w9WgXcQ")
print(f"   Found {len(languages)} languages:")
for lang in languages:
    auto = " (auto)" if lang["is_auto_generated"] else ""
    print(f"   - [{lang['language_code']}] {lang['language_name']}{auto}")

# 2. Fetch English auto-generated transcript
print("\n2. Fetching English (auto-generated) transcript...")
en_segments = fetcher.get_transcript("dQw4w9WgXcQ", "en", auto_generated=True)
print(f"   Got {len(en_segments)} segments")
print(f"   First: [{en_segments[0]['start']:.1f}s] {en_segments[0]['text']}")

# 3. Fetch Japanese manual transcript
print("\n3. Fetching Japanese (manual) transcript...")
ja_segments = fetcher.get_transcript("dQw4w9WgXcQ", "ja", auto_generated=False)
print(f"   Got {len(ja_segments)} segments")
print(f"   First: [{ja_segments[0]['start']:.1f}s] {ja_segments[0]['text']}")

print("\n" + "=" * 60)
print("✅ Hybrid solution works! No attestation required.")

Testing Complete Hybrid Solution

1. Listing available languages...
   Found 6 languages:
   - [en] English
   - [en] English (auto-generated) (auto)
   - [de-DE] German (Germany)
   - [ja] Japanese
   - [pt-BR] Portuguese (Brazil)
   - [es-419] Spanish (Latin America)

2. Fetching English (auto-generated) transcript...
   Got 52 segments
   First: [0.3s] [Music]

3. Fetching Japanese (manual) transcript...
   Got 60 segments
   First: [18.6s] 僕らは恋愛初心者じゃない

✅ Hybrid solution works! No attestation required.
